In [11]:
import json
import csv

def borda_score(ranks):
    points = {project: (5 - rank + 1) if rank > 0 else 0 for project, rank in ranks.items()}
    return points

def aggregate_votes(voters, num_projects=24):
    approval = {i: 0 for i in range(1, num_projects + 1)}
    k_approval = {i: 0 for i in range(1, num_projects + 1)}
    cumulative = {i: 0 for i in range(1, num_projects + 1)}
    rank = {i: 0 for i in range(1, num_projects + 1)}

    for voter in voters:
        inputs = voter["Inputs"]

        for i, vote in enumerate(inputs["voteA"], start=1):
            if vote:
                approval[i] += 1

        for i, vote in enumerate(inputs["voteB"], start=1):
            if vote:
                k_approval[i] += 1

        for i, vote in enumerate(inputs["voteD"], start=1):
            cumulative[i] += vote

        borda_scores = borda_score({i: rank for i, rank in enumerate(inputs["voteE"], start=1)})
        for i, score in borda_scores.items():
            rank[i] += score

    return approval, k_approval, cumulative, rank

project_data= {1: ['Planting Workshops at Oerlikon', 'Nord', 'Nature', '5000'], 2: ['Footpath Gardens around Train Stations', 'Nord', 'Nature', '10000'], 3: ['Kid Festival at Leutschenpark', 'Nord', 'Culture', '5000'], 4: ['Music Studio at Kulturbahnhof Affoltern', 'Nord', 'Culture', '10000'], 5: ['Safe Bike Paths around Irchel Park', 'Nord', 'Transportation', '5000'], 6: ['More Night Buses to Oerlikon', 'Nord', 'Transportation', '10000'], 7: ['Free Open Badi Space in Wollishofen', 'Süd', 'Nature', '5000'], 8: ['A Neighborhood Garden for Wiedikon', 'Süd', 'Nature', '10000'], 9: ['Environmental Film Session for Kids', 'Süd', 'Culture', '5000'], 10: ['Car-free Sunday Festivals on Mutschellenstrasse', 'Süd', 'Culture', '10000'], 11: ['Free Bike Fixing Workshops', 'Süd', 'Transportation', '5000'], 12: ['Car Sharing System for Young People', 'Süd', 'Transportation', '10000'], 13: ['Transforming City Spaces under Trees into Gardens', 'Ost', 'Nature', '5000'], 14: ['More Trees in Bellevue & Sechseläutenplatz', 'Ost', 'Nature', '10000'], 15: ['Plant Festival in the City Centre', 'Ost', 'Culture', '5000'], 16: ['Multicultural Festival at Sechseläutenplatz', 'Ost', 'Culture', '10000'], 17: ['Bike Lanes on Seefeldstrasse', 'Ost', 'Transportation', '5000'], 18: ['Speed bumps in the City and the Lake Front', 'Ost', 'Transportation', '10000'], 19: ['Bird Houses for Zurich-Höngg', 'West', 'Nature', '5000'], 20: ['Wet Biotope as Learning Garden in Frankental', 'West', 'Nature', '10000'], 21: ['Dingtheke: Community Things Exchange Library in Wipkingen', 'West', 'Culture', '5000'], 22: ['Sustainable Cooking Workshop with Kids', 'West', 'Culture', '10000'], 23: ['Public Bicycle Moving Trailer to be Borrowed', 'West', 'Transportation', '5000'], 24: ['Car-free Langstrasse', 'West', 'Transportation', '10000']}

with open('responses.json', 'r') as file:
    voters = json.load(file)

approval_votes, k_approval_votes, cumulative_votes, rank_votes = aggregate_votes(voters)

def write_csv(vote_dict, filename):
    sorted_votes = sorted(vote_dict.items(), key=lambda x: x[1], reverse=True)
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Rank', 'Votes', 'Id', 'Name', 'District', 'Category', 'Cost'])
        for rank, (project_id, total_votes) in enumerate(sorted_votes, start=1):
            project_info = project_data[project_id]
            writer.writerow([rank, total_votes, f"#{project_id}"] + project_info)

write_csv(approval_votes, 'outcome_lab_appr.csv')
write_csv(k_approval_votes, 'outcome_lab_kapp.csv')
write_csv(cumulative_votes, 'outcome_lab_cumu.csv')
write_csv(rank_votes, 'outcome_lab_rank.csv')
